In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

E:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
mnist = input_data.read_data_sets('MNIST_data/',one_hot=True)
x = tf.placeholder(tf.float32,[None,784])
y_ = tf.placeholder(tf.float32,[None,10])

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [4]:
x_image = tf.reshape(x,[-1,28,28,1])

In [5]:
# 截断正态分布初始化
def weight_variable(shape):
    initial = tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1,shape=shape)
    return tf.Variable(initial)

def conv2d(x,w):
    return tf.nn.conv2d(x,w,strides=[1,1,1,1],padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding="SAME")

In [6]:
#第一卷积层
W_conv1 = weight_variable([5,5,1,32])
b_conv1 = bias_variable([32])
h_conv1 = tf.nn.relu(conv2d(x_image,W_conv1)+b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

In [7]:
#第二卷积层
W_conv2 = weight_variable([5,5,32,64])
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1,W_conv2)+b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

In [8]:
#全连接层
W_fc1 = weight_variable([7*7*64,1024])
b_fc1 = bias_variable([1024])
h_pool2_flat = tf.reshape(h_pool2,[-1,7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat,W_fc1)+b_fc1)
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1,keep_prob)

In [9]:
#最后的全连接层
W_fc2 = weight_variable([1024,10])
b_fc2 = bias_variable([10])
y_conv = tf.matmul(h_fc1_drop,W_fc2)+b_fc2

In [10]:
#交叉熵
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_,logits=y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

In [11]:
correct_prediction = tf.equal(tf.argmax(y_conv,1),tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

In [12]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [13]:
for i in range(10000):
    batch = mnist.train.next_batch(50)
    if i%100 == 0:
        train_accuracy = accuracy.eval(feed_dict={x:batch[0],y_:batch[1],keep_prob:1.0})
        print("step %d,training accuracy %g" % (i,train_accuracy))
    train_step.run(feed_dict={x:batch[0],y_:batch[1],keep_prob:0.5})
#显存溢出
#print("test accuracy %g" % accuracy.eval(feed_dict={x:mnist.test.images,y_:mnist.test.labels,keep_prob:1.0}))

step 0,training accuracy 0.08
step 100,training accuracy 0.7
step 200,training accuracy 0.94
step 300,training accuracy 0.96
step 400,training accuracy 0.84
step 500,training accuracy 0.94
step 600,training accuracy 0.92
step 700,training accuracy 0.98
step 800,training accuracy 0.96
step 900,training accuracy 0.94
step 1000,training accuracy 0.94
step 1100,training accuracy 0.94
step 1200,training accuracy 0.98
step 1300,training accuracy 1
step 1400,training accuracy 0.98
step 1500,training accuracy 0.98
step 1600,training accuracy 0.94
step 1700,training accuracy 1
step 1800,training accuracy 0.92
step 1900,training accuracy 0.94
step 2000,training accuracy 0.94
step 2100,training accuracy 1
step 2200,training accuracy 1
step 2300,training accuracy 0.92
step 2400,training accuracy 0.98
step 2500,training accuracy 0.98
step 2600,training accuracy 0.96
step 2700,training accuracy 0.98
step 2800,training accuracy 0.96
step 2900,training accuracy 1
step 3000,training accuracy 0.98
step 

In [27]:
mean_value = 0.0
for i in range(mnist.test.labels.shape[0]):
    batch = mnist.test.next_batch(50)
    train_accuracy = sess.run(accuracy,feed_dict={x:batch[0],y_:batch[1],keep_prob:1.0})
    mean_value += train_accuracy
    if i%100 == 0:
        print("mean_value: %g train_accuracy: %g" %(mean_value,train_accuracy))
print('test_accuracy: %g' %(mean_value/mnist.test.labels.shape[0]))

mean_value: 0.96 train_accuracy: 0.96
mean_value: 100.1 train_accuracy: 1
mean_value: 199.18 train_accuracy: 0.98
mean_value: 298.22 train_accuracy: 1
mean_value: 397.42 train_accuracy: 1
mean_value: 496.58 train_accuracy: 1
mean_value: 595.62 train_accuracy: 0.96
mean_value: 694.76 train_accuracy: 1
mean_value: 793.82 train_accuracy: 0.94
mean_value: 892.88 train_accuracy: 1
mean_value: 992.1 train_accuracy: 1
mean_value: 1091.18 train_accuracy: 0.98
mean_value: 1190.28 train_accuracy: 0.98
mean_value: 1289.26 train_accuracy: 0.96
mean_value: 1388.54 train_accuracy: 1
mean_value: 1487.82 train_accuracy: 1
mean_value: 1586.74 train_accuracy: 0.98
mean_value: 1685.7 train_accuracy: 1
mean_value: 1784.94 train_accuracy: 0.96
mean_value: 1884.12 train_accuracy: 1
mean_value: 1983.2 train_accuracy: 1
mean_value: 2082.34 train_accuracy: 0.96
mean_value: 2181.38 train_accuracy: 0.98
mean_value: 2280.5 train_accuracy: 1
mean_value: 2379.62 train_accuracy: 0.98
mean_value: 2478.66 train_accura